### Import Required Libraries and Set Up Environment Variables

In [167]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [168]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [169]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url)
cme_response


<Response [200]>

In [170]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [171]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [172]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_filtered_df = cme_df[["activityID", "startTime", "linkedEvents"]]
cme_filtered_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [173]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_filtered_df = cme_filtered_df.dropna(subset=["linkedEvents"])
cme_filtered_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [174]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_filtered_df.index:
    # Get the corresponding row values
    activity_id = cme_filtered_df.loc[i, 'activityID']
    start_time = cme_filtered_df.loc[i, 'startTime']
    linked_events = cme_filtered_df.loc[i, 'linkedEvents']
    # Iterate over each dictionary in the list
    for event in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
                "activityID": activity_id,
                "startTime": start_time,
                "linkedEvent": event['activityID']  # Extract activityID from each event
            })
      
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.head()

,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T04:12:00-SEP-001


In [175]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        # Safely access 'activityID' key
        return input_dict['activityID']
    except (KeyError, TypeError) as e:
        # Log the error for debugging
        print(f"Error extracting 'activityID': {e}")
        return None  # Return None if extraction fails

# Example usage with one row from linkedEvents
example_event = {'activityID': '2013-05-04T04:52:00-IPS-001'}
extracted_activity_id = extract_activityID_from_dict(example_event)
extracted_activity_id

'2013-05-04T04:52:00-IPS-001'

In [176]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_filtered_df['GST_ActivityID'] = cme_filtered_df['linkedEvents'].apply(
    lambda x: extract_activityID_from_dict(x[0]) if isinstance(x, list) and len(x) > 0 else None
)
cme_filtered_df.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}],2013-05-04T04:52:00-IPS-001
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}],2013-05-07T04:37:00-IPS-001
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}],2013-05-12T23:30:00-IPS-001
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...,2013-05-13T01:53:00-FLR-001
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...,2013-05-13T15:40:00-FLR-001


In [177]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_filtered_df = cme_filtered_df.dropna(subset=["GST_ActivityID"])

In [178]:
# print out the datatype of each column in this DataFrame:
cme_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1023 non-null   object
 1   startTime       1023 non-null   object
 2   linkedEvents    1023 non-null   object
 3   GST_ActivityID  1023 non-null   object
dtypes: object(4)
memory usage: 72.2+ KB


In [179]:
print(cme_filtered_df.columns)

Index(['activityID', 'startTime', 'linkedEvents', 'GST_ActivityID'], dtype='object')


In [180]:
# Convert the 'GST_ActivityID' column to string format
# Ensure all NaN/None values are handled by replacing them with an empty string first
cme_filtered_df['GST_ActivityID'] = (
    cme_filtered_df['GST_ActivityID']
    .fillna('')                      # Replace NaN with empty string
    .astype('string')                # Convert to string type
)

# Convert startTime_CME to datetime format
cme_filtered_df['startTime'] = pd.to_datetime(cme_filtered_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme_filtered_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents if it exists
if 'linkedEvents' in cme_filtered_df.columns:
    cme_filtered_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print(cme_filtered_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1023 non-null   object             
 1   startTime_CME   1023 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1023 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 64.3+ KB
None


In [181]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
gst_related_cmes = cme_filtered_df[cme_filtered_df['GST_ActivityID'].str.contains('GST', na=False)]
# gst_related_cmes.reset_index(drop=True, inplace=True)
gst_related_cmes.head()

,cmeID,startTime_CME,GST_ActivityID
26,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1358,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
1359,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001
1361,2015-09-06T23:30:00-CME-001,2015-09-06 23:30:00+00:00,2015-09-11T06:00:00-GST-001
1362,2015-09-07T10:36:00-CME-001,2015-09-07 10:36:00+00:00,2015-09-11T06:00:00-GST-001


### GST Data

In [182]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
# print("GST URL:", gst_url)

In [183]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)
gst_response

<Response [200]>

In [184]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [185]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_filtered_df = gst_df[["gstID", "startTime", "linkedEvents"]]
gst_filtered_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [186]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_filtered_df = gst_filtered_df.dropna(subset=['linkedEvents'])
gst_filtered_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [187]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_filtered_df = gst_filtered_df.explode('linkedEvents').reset_index(drop=True)
gst_filtered_df = gst_filtered_df.dropna(subset=['linkedEvents'])
gst_filtered_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [188]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_filtered_df['CME_ActivityID'] = gst_filtered_df['linkedEvents'].apply(
    lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None
)
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_filtered_df = gst_filtered_df[gst_filtered_df['CME_ActivityID'].notnull()]

gst_filtered_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [189]:
# Convert the 'CME_ActivityID' column to string format 
gst_filtered_df['CME_ActivityID'] = gst_filtered_df['CME_ActivityID'].astype('string')
# Convert the 'gstID' column to string format 
gst_filtered_df['gstID'] = gst_filtered_df['gstID'].astype('string')
# Convert startTime to datetime format  
gst_filtered_df['startTime'] = pd.to_datetime(gst_filtered_df['startTime'])
# Rename startTime to startTime_GST 
gst_filtered_df.rename(columns={"startTime": "startTime_GST"}, inplace=True)
# Drop linkedEvents
gst_filtered_df.drop(columns=["linkedEvents"], inplace=True)
# Verify that all steps were executed correctly
gst_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 4.9 KB


In [190]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
filtered_gst_cmes = gst_filtered_df[gst_filtered_df['CME_ActivityID'].str.contains('CME', na=False)]
filtered_gst_cmes.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [191]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(filtered_gst_cmes, gst_related_cmes, left_on='gstID', right_on='GST_ActivityID')
merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
2,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001
3,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
4,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001


In [192]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           11 non-null     string             
 1   startTime_GST   11 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  11 non-null     string             
 3   cmeID           11 non-null     object             
 4   startTime_CME   11 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  11 non-null     string             
dtypes: datetime64[ns, UTC](2), object(1), string(3)
memory usage: 660.0+ bytes


### Computing the time it takes for a CME to cause a GST

In [193]:
# Compute the time difference between startTime_GST and startTime_CME
# by creating a new column called 'timeDiff'.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

# Display the first few rows of the updated DataFrame
merged_df.head()


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001,4 days 12:48:00
2,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001,4 days 07:36:00
3,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001,4 days 12:48:00
4,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001,4 days 07:36:00


In [194]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_diff_stats = merged_df['timeDiff'].describe()
time_diff_stats

count                           11
mean     3 days 17:25:05.454545454
std      1 days 03:53:04.735529226
min                1 days 10:12:00
25%                3 days 19:24:00
50%                4 days 06:30:00
75%                4 days 07:36:00
max                4 days 12:48:00
Name: timeDiff, dtype: object

### Exporting data in csv format

In [195]:
# Export data to CSV without the index
merged_df.to_csv("gst_cme_data.csv", index=False)
